## Portfolios and Portfolio Groups

Portfolios are the main containers for transactions and holdings, and come in multiple forms based on hierarchy and use case. The basic portfolio types are :
- Transaction Portfolios
- Reference Portfolios
- Derived Portfolios

These portfolios can then be grouped, where portfolio operations on the group are applied to an aggregated set of portfolios in the hierarchy.

This example will show how to :
1. Create Transactions Portfolios
2. Create Derived Portfolios
3. (add/get/delete portfolio properties and details)
4. Create portfolio Groups
5. Add portfolios to a Group
6. Add a group to a group (creation of subgroups)
7. Modify groups
8. Remove portfolios and subgroups from groups
9. Delete portfolios
10. Delete groups
11. Get all the commands applied to a group or portfolio
12. List all the groups and portfolios in a scope

Initialise our environment and connect to LUSID

In [1]:
# Import LUSID
import lusid
import lusid.models as models
import lusid_sample_data as import_data
# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import uuid
import printer as prettyprint
from datetime import datetime
import pandas as pd
import numpy as np
import os
from msrest.authentication import BasicTokenAuthentication

# Authenticate our user and create our API client
client = import_data.authenticate_secrets()

print ('LUSID Environment Initialised')
print ('LUSID version : ', client.api_version)

LUSID Environment Initialised
LUSID version :  0.9.127


First we start by creating a couple of default portfolios, we will put them in two different scopes to demonstrate the separation between scopes and the usage of different scopes within portfolio groups.

In [3]:
# Create a transaction Portfolio - in two scopes one for groups use one for list in scopes use
scope1 = "python_portfolios_main2"
scope2 = "python_portfolios_second2"
guid = str(uuid.uuid4())
effective_date = datetime(2018, 1, 1, tzinfo=pytz.utc)

request1 = models.CreateTransactionPortfolioRequest(
    "portfolio-{0}".format(guid),
    "id-{0}".format(guid),
    base_currency="GBP",
    created=effective_date
)

guid = str(uuid.uuid4())  # reset guid for second portfolio
request2 = models.CreateTransactionPortfolioRequest(
    "portfolio-{0}".format(guid),
    "id-{0}".format(guid),
    base_currency="GBP",
    created=effective_date
)
result1 = client.create_portfolio(scope1, request1)
result2 = client.create_portfolio(scope2, request2)
portfolio_id1 = result1.id.code
portfolio_id2 = result2.id.code

prettyprint.portfolio_response(result1)
prettyprint.portfolio_response(result2)


Portfolio Created
Scope: python_portfolios_main2
Code: id-20f606cd-8e01-4ba6-b24f-4b4e820ce308
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-11 11:25:03.878979+00:00

Portfolio Created
Scope: python_portfolios_second2
Code: id-9d750652-601a-4d3f-b0d5-69d80a7ff603
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-11 11:25:04.291559+00:00



We then create one derived portfolio from the first transaction portfolio created.

In [106]:
# Create derived transaction portfolio from portfolio 1
# guid = str(uuid.uuid4())
# derived_portfolio_request = models.CreateDerivedTransactionPortfolioRequest(
#         display_name="derived_portfolio",
#         code="derived_portfolio-{0}".format(guid),
#         parent_portfolio_id=models.ResourceId(
#             scope=scope1,
#             code=portfolio_id1),
#         description="derived_portfolio_description",
#         created=effective_date)

# derived_portfolio = client.create_derived_portfolio(scope1, derived_portfolio_request)

# prettyprint.portfolio_response(derived_portfolio)

Derived Portfolio Created
Scope: python_portfolios_main2
Code: derived_portfolio-4d6b128a-72b9-4556-a1b0-1911ac1c52db
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-08 16:21:27.544610+00:00

Parent Portfolio Details
Code: id-5f503b61-348f-405b-a504-af6a726f7725


In [83]:
# ------ LEAVE FOR LAST --------
# Delete portfolio details

# Upsert new details

# Get portfolio details

# Get portfolio properties

# Upsert portfolio properties

# Delete properties
# ------------------------------


# derived_portfolio_details = client.get_portfolio(scope1, derived_portfolio.id.code)
# print('Portfolio with ID is derived? : ', derived_portfolio_details.is_derived)
# Update Portfolio Definition -- new name
# update_request = models.update_portfolio_request(
#     scope1, derived_portfolio.id.code, display_name="derived_portfolio_name")



Next we can create some portfolio groups, add the portfolios to the groups, add one of the groups to the other to create a subgroup, and subsequently update its name to reflect this change in hierarchy.
Note that the second group is in a different scope to the first, it is entirely possible to combine portfolios and groups in different scopes, keep in mind that group-wide operations will still only apply to the specified scope, and will not affect portfolios/subgroups in different scopes.

In [4]:
# Create a portfolio group -- create two
guid = str(uuid.uuid4())
group_request1 = models.CreatePortfolioGroupRequest(
    id="portfolio_group-{0}".format(guid),
    display_name="portfolio_group1_name",
    values=None)

guid = str(uuid.uuid4())
group_request2 = models.CreatePortfolioGroupRequest(
    id="portfolio_group-{0}".format(guid),
    display_name="portfolio_group2_name",
    values=None)

portfolio_group1 = client.create_portfolio_group(scope1, group_request1)
portfolio_group2 = client.create_portfolio_group(scope2, group_request2)

prettyprint.portfolio_group_response(portfolio_group1, 'created')
prettyprint.portfolio_group_response(portfolio_group2, 'created')



Portfolio Group Created
Name: portfolio_group1_name
Scope: python_portfolios_main2
Code: portfolio_group-c2acdbe5-5869-46f7-894a-07f847602463
Portfolios Inside Group: 


Portfolio Group Created
Name: portfolio_group2_name
Scope: python_portfolios_second2
Code: portfolio_group-32dd68b6-8580-4529-8918-0942627abd37
Portfolios Inside Group: 




In [5]:
# Add portfolios to groups

client.add_portfolio_to_group(portfolio_group1.id.scope, portfolio_group1.id.code, portfolio_id=result1.id)
client.add_portfolio_to_group(portfolio_group2.id.scope, portfolio_group2.id.code, portfolio_id=result2.id)


In [6]:
# Add group to group
client.add_sub_group_to_group(portfolio_group1.id.scope, portfolio_group1.id.code, portfolio_group2.id)

In [7]:
# Update group -- since group 2 is now a subgroup, lets update the name to reflect this
group_update_request = models.UpdatePortfolioGroupRequest(display_name="Group1_Subgroup")
client.update_portfolio_group(portfolio_group2.id.scope, portfolio_group2.id.code, group_update_request)

We can now compare two methods of obtaining groups from LUSID. The first is `get portfolio group` while the second is `get portfolio group expansion`. The difference here is that the first method will return only information about the specified group, including the ids of the portfolios it contains, but not any details of its subgroups. The second method will not only return this basic information, but also the full details of a portfolio, a list of all the subgroups contained in the group with all the details of portfolios contained within each subgroup as well. 


In [8]:
# Get portfolio group
updated_group1 = client.get_portfolio_group(portfolio_group1.id.scope, portfolio_group1.id.code, as_at=datetime.utcnow().replace(tzinfo=pytz.utc))
original_group1 = client.get_portfolio_group(portfolio_group1.id.scope, portfolio_group1.id.code, as_at=portfolio_group1.version.as_at_date)

prettyprint.portfolio_group_response(updated_group1, 'none')
prettyprint.portfolio_group_response(original_group1, 'none')

Portfolio Group
Name: portfolio_group1_name
Scope: python_portfolios_main2
Code: portfolio_group-c2acdbe5-5869-46f7-894a-07f847602463
Portfolios Inside Group: 
id-20f606cd-8e01-4ba6-b24f-4b4e820ce308


Portfolio Group
Name: portfolio_group1_name
Scope: python_portfolios_main2
Code: portfolio_group-c2acdbe5-5869-46f7-894a-07f847602463
Portfolios Inside Group: 




In [89]:
# Get full expansion of a portfolio group
expansion = client.get_portfolio_group_expansion(portfolio_group1.id.scope, portfolio_group1.id.code)

prettyprint.expanded_portfolio_group_response(expansion)

Portfolio Group Full Details : 
Scope: python_portfolios_main2
Code: portfolio_group-7d380896-30c4-4884-9bbc-6168821c0632


Portfolios Inside Group: 
Portfolio Created
Scope: python_portfolios_main2
Code: id-ad3e46ad-93b8-4d8d-a42c-53f842db2fa1
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-08 16:20:06.539599+00:00



Subgroups Inside Group: 
Name: Group1_Subgroup
Scope: python_portfolios_second2
Code: portfolio_group-2fb2ca2c-15ab-45d0-b304-45db99c1d866


Portfolios Inside SubGroup: 
Portfolio Created
Scope: python_portfolios_second2
Code: id-13fd72a6-d89e-4b42-8481-bdb2f6b0be5b
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-08 16:20:03.525864+00:00





Suppose we wish to revert these changes, we have to explicitly delete each component step by step. Our Subgroup contains one portfolio, we're going to remove it from that group, remove the subgroup from the main group, and then delete it as it is no longer needed. If a subgroup is empty, it is not necessary to remove it before deleting it completely, however for example purposes we will follow each step.

In [90]:
#1. remove portfolio from subgroup
portfolio_removal = client.delete_portfolio_from_group(portfolio_group2.id.scope, portfolio_group2.id.code, result2.id.scope, result2.id.code)
prettyprint.portfolio_group_response(portfolio_removal, 'none')


Portfolio Group
Name: Group1_Subgroup
Scope: python_portfolios_second2
Code: portfolio_group-2fb2ca2c-15ab-45d0-b304-45db99c1d866
Portfolios Inside Group: 




In [91]:
#2. Remove group from group
group_removal = client.delete_sub_group_from_group(portfolio_group1.id.scope, portfolio_group1.id.code, portfolio_group2.id.scope, portfolio_group2.id.code)
prettyprint.portfolio_group_response(group_removal, 'none')
print('Remaining Sub Groups : ', group_removal.sub_groups)

Portfolio Group
Name: portfolio_group1_name
Scope: python_portfolios_main2
Code: portfolio_group-7d380896-30c4-4884-9bbc-6168821c0632
Portfolios Inside Group: 
id-ad3e46ad-93b8-4d8d-a42c-53f842db2fa1


Remaining Sub Groups :  []


In [92]:
#3. Delete group
group_deletion = client.delete_portfolio_group(portfolio_group2.id.scope, portfolio_group2.id.code)

We can now check what operations have been applied to each of our groups, including an already deleted group, by using the `get portfolio group commands` method, as below.

In [93]:
# Get commands
group1_commands = client.get_portfolio_group_commands(portfolio_group1.id.scope, portfolio_group1.id.code)
group2_commands = client.get_portfolio_group_commands(portfolio_group2.id.scope, portfolio_group2.id.code)

prettyprint.group_commands(group1_commands, portfolio_group1.display_name)
prettyprint.group_commands(group2_commands, portfolio_group2.display_name)

Commands Applied To Group portfolio_group1_name
Number of commands : 4
Description : Create portfolio group
At Time : 2019-02-08T16:20:08.6276890+00:00
Description : Add portfolio to group
At Time : 2019-02-08T16:20:10.1777050+00:00
Description : Add subgroup to group
At Time : 2019-02-08T16:20:11.0048760+00:00
Description : Delete subgroup from group
At Time : 2019-02-08T16:20:14.6326580+00:00
Commands Applied To Group portfolio_group2_name
Number of commands : 5
Description : Create portfolio group
At Time : 2019-02-08T16:20:08.9516710+00:00
Description : Add portfolio to group
At Time : 2019-02-08T16:20:10.5980010+00:00
Description : Update portfolio group
At Time : 2019-02-08T16:20:11.4001070+00:00
Description : Delete portfolio from group
At Time : 2019-02-08T16:20:13.4364980+00:00
Description : Delete portfolio group
At Time : 2019-02-08T16:20:17.0383850+00:00


We can now verify that the group has indeed been deleted from our scope by getting all groups in each of our scopes, the second of which should be empty now.

In [ ]:
# List groups in scope - show one scope with no groups

If we list all the portfolios in our scopes we will see that they are all still there. There may be extra portfolios not created in this example, if this is the case, we can delete all these extra portfolios and keep only those we are interested in.

In [108]:

scope1_portfolios = client.list_portfolios_for_scope(scope1)
scope2_portfolios = client.list_portfolios_for_scope(scope2)

print('Portfolios remaining in scope ', scope1, ' :')
for portfolio in scope1_portfolios.values:
    prettyprint.portfolio_response(portfolio)
    print('\n')
    
print('Portfolios remaining in scope ', scope2, ' :')
for portfolio in scope2_portfolios.values:
    prettyprint.portfolio_response(portfolio)
    print('\n')

Portfolios remaining in scope  python_portfolios_main2  :
Derived Portfolio Created
Scope: python_portfolios_main2
Code: derived_portfolio-4d6b128a-72b9-4556-a1b0-1911ac1c52db
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-08 16:21:27.544610+00:00

Parent Portfolio Details
Code: id-5f503b61-348f-405b-a504-af6a726f7725


Derived Portfolio Created
Scope: python_portfolios_main2
Code: derived_portfolio-b233bfc4-8435-4320-8389-0e959ab930ff
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-08 16:20:06.539599+00:00

Parent Portfolio Details
Code: id-ad3e46ad-93b8-4d8d-a42c-53f842db2fa1


Portfolio Created
Scope: python_portfolios_main2
Code: id-ad3e46ad-93b8-4d8d-a42c-53f842db2fa1
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-08 16:20:06.539599+00:00



Portfolio Created
Scope: python_portfolios_main2
Code: id-5f503b61-348f-405b-a504-af6a726f7725
Portfolio Effective From: 2018-01-01 00:00:0

The cells below allow us to delete the extra portfolios in the scope

In [109]:
# We can delete all the portfolios in the scope that aren't ours now.
def extractIds(portfolioList):
    ids = []
    for folio in portfolioList:
        #print(folio.id.scope, folio.id.code)
        ids.append((folio.id.scope, folio.id.code))
    return ids
    
def returnNotMatches(a, b):
    return [x for x in a if x not in b]
# first combine all the portfolios in our two scopes
all_portfolios_in_scopes = extractIds(scope1_portfolios.values + scope2_portfolios.values)

# then combine the portfolios we're interested in keeping into a list
portfolios_to_keep = extractIds([result1, result2, derived_portfolio])

# we can now exclude portfolios to keep from the full list
portfolios_to_delete = returnNotMatches(all_portfolios_in_scopes, portfolios_to_keep)


In [110]:
for portfolio in portfolios_to_delete:
    response = client.delete_portfolio(portfolio[0], portfolio[1])
    print ('Portfolio deleted : ', portfolio[0], portfolio[1])
    print ('At time : ', str(response.as_at))

Portfolio deleted :  python_portfolios_main2 derived_portfolio-b233bfc4-8435-4320-8389-0e959ab930ff
At time :  2019-02-08 16:21:51.324748+00:00
Portfolio deleted :  python_portfolios_main2 id-ad3e46ad-93b8-4d8d-a42c-53f842db2fa1
At time :  2019-02-08 16:21:51.715196+00:00
Portfolio deleted :  python_portfolios_second2 id-13fd72a6-d89e-4b42-8481-bdb2f6b0be5b
At time :  2019-02-08 16:21:52.037880+00:00


If we now re-list the portfolios in our scopes, we will see that only our two original portfolios remain, and one derived portfolio.

In [111]:
print(client.list_portfolios_for_scope(scope1))

{'additional_properties': {}, 'values': [<lusid.models.portfolio.Portfolio object at 0x7fb37eeb04e0>, <lusid.models.portfolio.Portfolio object at 0x7fb37eeb08d0>], 'href': 'https://api.lusid.com/api/portfolios/python_portfolios_main2', 'count': 2, 'links': [<lusid.models.link.Link object at 0x7fb37eeb0a90>]}
